In [5]:
# pip3 install cloudscraper beautifulsoup4
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import time
import re
from deep_translator import GoogleTranslator
from unidecode import unidecode
import json


In [2]:
def hanNopHoSo(soup):
    deadline_div = soup.find('div', class_='job-detail__info--deadline')
    # Nếu tìm thấy, lấy phần thông tin về hạn nộp hồ sơ
    if deadline_div:
        deadline = deadline_div.text.strip()
        deadline = deadline.lstrip("Hạn nộp hồ sơ: ").strip()
        return str(deadline)
    return None

In [3]:
def linhVucCongTy(soup):
    # Tìm <div> có class "company-title" chứa "Lĩnh vực:" bằng get_text
    company_title_div = next(
        (div for div in soup.find_all('div', class_="company-title") if "Lĩnh vực:" in div.get_text(strip=True)),
        None
    )

    # Nếu tìm thấy, lấy thông tin từ <div class="company-value">
    if company_title_div:
        company_value_div = company_title_div.find_next('div', class_="company-value")
        if company_value_div:
            company_value = company_value_div.text.strip()
            return company_value
        return None
    return None

In [4]:
def thongTinChung(soup):
    # Danh sách các tiêu đề cần tìm
    titles_to_find = ["Cấp bậc", "Số lượng tuyển", "Hình thức làm việc", "Giới tính"]
    array = []
    # Duyệt qua từng tiêu đề và tìm giá trị tương ứng
    for title in titles_to_find:
        title_div = soup.find('div', class_="box-general-group-info-title", string=title)
        if title_div:
            value_div = title_div.find_next('div', class_="box-general-group-info-value")
            if value_div:
                value = value_div.text.strip()
                array.append(value)
    if not array:
        return None
    return array

In [5]:
def thongTinTop(soup):
    # Danh sách các tiêu đề cần tìm
    titles_to_find = ["Mức lương", "Địa điểm", "Kinh nghiệm"]
    array = []
    # Duyệt qua từng tiêu đề và tìm giá trị tương ứng
    for title in titles_to_find:
        title_div = soup.find('div', class_='job-detail__info--section-content-title', string=title)
        if title_div:
            value_div = title_div.find_next('div', class_="job-detail__info--section-content-value")
            if value_div:
                value = value_div.text.strip()
                array.append(value)
    if not array:
        return None
    return array

In [6]:
def convert_to_camel_case(text):
    # Chuyển tất cả chữ về thường
    text = unidecode(text.lower())
    # Tách các từ dựa trên khoảng trắng
    words = text.split()
    # Viết hoa chữ cái đầu của từng từ và nối chúng lại
    camel_case = ''.join(word.capitalize() for word in words)
    return camel_case

In [7]:
def linhVuc(soup):
    jobs = soup.find('div', class_="job-tags")
    if jobs:
        # Tách nội dung các phần tử con trong 'job-tags' và ngăn cách bằng dấu ";"
        job_list = jobs.get_text(separator='; ', strip=True)
        return str(job_list)
    return None

In [8]:
def thoiGianLamViec(soup):
    # Tìm thẻ <h3> với nội dung "Mô tả công việc"
    job_desc_title = soup.find(['h2', 'h3'], string='Thời gian làm việc')

    # Nếu tìm thấy, lấy thông tin mô tả công việc từ thẻ sau đó
    if job_desc_title:
        content_div = job_desc_title.find_next('div', class_='job-description__item--content')
        if content_div:
            job_desc = content_div.get_text(separator='; ', strip=True)  # Dùng separator để ngăn cách các câu
            return job_desc
    return None

In [9]:
def cachThucUngTuyen(soup):
    # Tìm thẻ <h3> với nội dung "Mô tả công việc"
    job_desc_title = soup.find(['h2', 'h3'], string="Cách thức ứng tuyển")

    # Nếu tìm thấy, lấy thông tin mô tả công việc từ thẻ sau đó
    if job_desc_title:
        content_div = job_desc_title.find_next('div', class_='job-description__item--content')
        if content_div:
            job_desc = content_div.get_text(separator=' ', strip=True)  # Dùng separator để ngăn cách các câu
            return job_desc
    return None

In [3]:

def diaChi(soup):
    # Tìm thẻ <h3> với nội dung "Mô tả công việc"
    job_desc_title = soup.find(['h2', 'h3'], string="Địa điểm làm việc")
    tinh_array = []
    quan_array = []
    con_lai_array = []
    # Nếu tìm thấy, lấy thông tin mô tả công việc từ thẻ sau đó
    if job_desc_title:
        content_div = job_desc_title.find_next('div', class_='job-description__item--content')
        if content_div:
        # Tìm thẻ <a> với id="toggle-more-job-addresses"
            a_tag = content_div.find('a', id="toggle-more-job-addresses")
            if a_tag:  # Nếu tìm thấy thẻ <a>
                a_tag.decompose()  # Xóa thẻ <a>
            # div_tag = content_div.find('div', id="more-job-addresses")
            # if div_tag:  # Nếu tìm thấy thẻ <a>
            #     div_tag.decompose()  # Xóa thẻ <a>
            job_desc = content_div.text.strip()
            # print(f"Địa điểm làm việc: {job_desc}")
            # job_desc = job_desc.lstrip("-").strip()
            job_desc = job_desc.split("\n")
            for temp in job_desc:
                temp = temp.lstrip("-").strip()
                address = temp.split(':')
                address1 = temp.split(',')
                if len(address) > 1:
                    if len(address1) > 1:
                        tinh = address[0]
                        quan = address1[-1]
                        con_lai = ','.join(address1[:-1])
                        con_lai = con_lai.split(':')
                        # print(con_lai, quan, tinh)
                        con_lai_array.append(con_lai[-1])
                        quan_array.append(quan)
                        tinh_array.append(tinh)
                    else:
                        tinh = address[0]
                        quan = address[1]
                        # print(con_lai, quan, tinh)
                        con_lai_array.append("")
                        quan_array.append(quan)
                        tinh_array.append(tinh)
                else:
                    tinh = address[0]
                    con_lai_array.append("")
                    quan_array.append("")
                    tinh_array.append(tinh)
    # array.append(con_lai_array)
    # array.append(quan)
    filtered = [
    (x, y, z)
    for x, y, z in zip(con_lai_array, quan_array, tinh_array)
    if any([x, y, z])  # Giữ nếu ít nhất một phần tử không rỗng
    ]

    # Kết quả tách riêng từng mảng sau khi lọc
    con_lai_array, quan_array, tinh_array = zip(*filtered)
    if not tinh_array:
        return None
    return ';'.join(con_lai_array), ';'.join(quan_array), '; '.join(tinh_array)

In [7]:
diaChi(soup)

(' 22/14 Phan Văn Hớn, phường Tân Thới Nhất', ' Quận 12', 'Hồ Chí Minh')

In [11]:
def clean_lines(text):
    # Loại bỏ các ký tự không mong muốn ở đầu mỗi dòng
    cleaned_text = "\n".join(line.lstrip("-+●•.").strip() for line in text.splitlines())
    return cleaned_text

In [12]:
def thongTinCongViec(soup):
    titles_to_find = ["Mô tả công việc", "Yêu cầu ứng viên", "Quyền lợi"]
    array = []
    for title in titles_to_find:
        # Tìm thẻ <h3> với nội dung "Mô tả công việc"
        job_desc_title = soup.find(['h2', 'h3'], string=title)
        if job_desc_title:
            content_div = job_desc_title.find_next('div', class_='job-description__item--content')
            if content_div:
                # Lấy toàn bộ nội dung HTML bên trong, loại bỏ thẻ <div> ngoài cùng
                inner_html = content_div.decode_contents().strip()
                array.append(inner_html)
            else:
                array.append(None)
    if not array:
        return None
    return array

In [13]:
def get_url(soup):
    canonical_tag = soup.find('link', rel='canonical')
    # Lấy giá trị của thuộc tính href
    if canonical_tag and 'href' in canonical_tag.attrs:
        canonical_url = canonical_tag['href']
        return str(canonical_url)
    return None

In [14]:
def get_name_company(soup):
    company_name = soup.find(['h1' ,'h2', 'h3'], class_="company-name-label")
    if company_name:
        name = company_name.get_text(separator=' ', strip=True)
        return name
    return None

In [15]:
def get_link_company(soup):
    # Tìm thẻ <a> bên trong <h2>
    link = soup.find(['h1' ,'h2', 'h3'], class_='company-name-label').find('a', class_='name')['href']
    if link:
        return link
    return None

In [16]:
def get_id_company(soup):
    temp = get_link_company(soup)
    if temp:
        # Regex
        pattern = r"\/(\d+)(?=\.html)"
        match = re.search(pattern, temp)
        if match:
            return match.group(1)
    return None

In [17]:
def get_heading(soup):
    heading1 = soup.find('h1', class_="job-detail__info--title")
    if heading1:
        content = heading1.get_text(separator=' ', strip=True)
        return str(content)
    return None

In [18]:
def remove_leading_dash(text):
    if text.strip().startswith('-'):
        return text.lstrip().removeprefix('-').strip()
    
    # Trả về chuỗi gốc nếu không bắt đầu bằng '-'
    return text.strip()

In [19]:
def replace_semicolons(input_string):
    # Loại bỏ chuỗi nếu chỉ chứa dấu ;
    if input_string.strip(';') == '':
        return ''

    # Thay nhiều dấu "," liên tiếp bằng một dấu ","
    input_string = input_string.replace(',,', ',')
    while ',,' in input_string:  # Đảm bảo thay hết các trường hợp
        input_string = input_string.replace(',,', ',')

    # Xóa dấu "," ở cuối chuỗi
    input_string = input_string.rstrip(',')
    input_string = input_string.rstrip(';')
    input_string = input_string.rstrip('-')
    input_string = input_string.rstrip('–')
    input_string = input_string.rstrip('.')
    input_string = remove_leading_dash(input_string)
    return input_string


In [20]:
def process_district(dist):
    if 'q.' in dist:
        dist = dist.replace('q.', 'quận')
    elif not 'quận' in dist and not 'tp' in dist and not 'tp.' in dist and not 'thành phố' in dist:
        dist = 'quận ' + dist
    return dist

In [21]:
def processing_address(text, district):
     # Tách các địa chỉ và các quận/huyện dựa trên dấu chấm phẩy
    text_parts = text.split(';')
    district_parts = district.split(';')

    # Xử lý từng cặp địa chỉ và quận/huyện
    processed_addresses = []
    for addr, dist in zip(text_parts, district_parts):
        dist = dist.lower().strip()  # Chuẩn hóa quận/huyện để so sánh
        addr_lower = addr.lower().strip()            
        dist_trans = GoogleTranslator(source='vi', target='en').translate(process_district(dist))
        dist_trans = dist_trans.lower().strip()
        pattern = rf"(?<=\d)[a-zA-Z]{dist.replace(' ', '')}|phường\s*{dist.replace(' ', '')}|{dist.replace(' ', '')}\)"
        pattern1 = rf'(Q\.|Quận|Q\d+|Q\s*{dist}|Huyện|Thị xã).*$'
        # pattern3 = rf"[, \-\–]{dist.replace(' ', '')}"
        pattern4 = r'[, \-\–]\s*(Thủ Đức|TP|TP\.|thành phố|hn|hcm|hà nội|ho chi minh|ha noi|hochiminh|hanoi)\s*.*,?$'
        if re.search(pattern1, addr_lower, flags=re.IGNORECASE):  # Kiểm tra sự tồn tại của mẫu
            addr = re.sub(rf'(Q\.|Quận|Q\d+|Q\s*{dist}|Huyện|Thị xã).*$', '', addr, flags=re.IGNORECASE).strip()  # Xóa phần dư "Quận ..."
            # Kiểm tra điều kiện
            if re.search(pattern4, addr, flags=re.IGNORECASE):
                addr = re.sub(pattern4, '', addr, flags=re.IGNORECASE).strip()
            processed_addresses.append(replace_semicolons(addr))
        elif dist_trans in addr_lower:
            last_index = addr_lower.rfind(dist_trans)
            # Xóa từ vị trí district đến hết
            addr = addr[:last_index].strip()
            # Kiểm tra điều kiện
            if re.search(pattern4, addr, flags=re.IGNORECASE):
                addr = re.sub(pattern4, '', addr, flags=re.IGNORECASE).strip()
            processed_addresses.append(replace_semicolons(addr))
        elif unidecode(dist) in unidecode(addr_lower) and not re.search(pattern, addr_lower.replace(" ", ""), flags=re.IGNORECASE):
            #  and re.search(pattern3, addr_lower.replace(" ", ""), flags=re.IGNORECASE)
            # Tìm vị trí xuất hiện cuối cùng của district
            last_index = unidecode(addr_lower).rfind(unidecode(dist))
            # Xóa từ vị trí district đến hết
            addr = addr[:last_index].strip()
            # Kiểm tra điều kiện
            if re.search(pattern4, addr, flags=re.IGNORECASE):
                addr = re.sub(pattern4, '', addr, flags=re.IGNORECASE).strip()
            processed_addresses.append(replace_semicolons(addr))
        elif('hoạt động trên địa bàn' not in addr_lower and 'cân đối theo khu vực quản lý' not in addr_lower and 'trao đổi khi phỏng vấn' not in addr_lower):
            # Kiểm tra điều kiện
            if re.search(pattern4, addr, flags=re.IGNORECASE):
                addr = re.sub(pattern4, '', addr, flags=re.IGNORECASE).strip()
            processed_addresses.append(replace_semicolons(addr))
    return '; '.join(processed_addresses)


In [22]:
def process_ChuyenMon(text):
    if not text:
        return None
    data =  re.sub(r"^chuyên môn[: ]+", "", text, flags=re.IGNORECASE)
    data_split = data.split(';')
    return str(data_split[0])


In [23]:
def convert_number(value):
    # try:
        if not value:
            return None
        number = float(value.replace(",", ""))
        # Kiểm tra nếu là số nguyên
        if number.is_integer():
            return int(number)
        else:
            return number  # Trả về float
    # except Exception:
    #     return value  # Trả về giá trị gốc nếu có lỗi

In [24]:
def load_df():
    return pd.read_excel(r'C:\Users\My_Pc\Desktop\Thư Viện Pháp Luật final\Crawl tuyển dụng\Code\data_id\coquanhanhchinh.xlsx', engine='openpyxl')


In [25]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhThanh(name, df):
    # print(name)
    result = df[
        df['UnitName'].str.lower().str.contains(name.lower()) &
        (df['ParentId'] == 0) &
        (df['Level'] == 1)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        # parent_id_value = result.iloc[0]['ParentId']
        # level_value = result.iloc[0]['Level']
        name_value = result.iloc[0]['UnitName']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value
    return None, None


In [26]:
# Tìm kiếm với nhiều điều kiện
def find_id_tinhQuanHuyen(name, ParentId, df):
    if not name:
        return None, None
    name = name.replace("Quận 2", "Thủ Đức")
    result = df[
        df['UnitName'].str.lower().str.contains(name.lower()) &
        (df['ParentId'] == ParentId) &
        (df['Level'] == 2)
    ]

    if not result.empty:
        id_value = result.iloc[0]['Id']
        name_value = result.iloc[0]['UnitName']
        # print(f"Id: {id_value}, ParentId: {parent_id_value}, Level: {level_value}")
        return id_value, name_value
    return None, None


In [27]:
def process_address_data(dia_chi, df):
    # Tách các thành phần địa chỉ
    soNha = replace_semicolons(processing_address(str(dia_chi[0]), replace_semicolons(str(dia_chi[1])))).strip().split(';')
    quanHuyen = replace_semicolons(str(dia_chi[1])).split(';')
    tinhThanh = str(dia_chi[2]).split(';')
    # data_dict = 
    # Tìm độ dài tối đa của 3 mảng
    max_length = max(len(soNha), len(quanHuyen), len(tinhThanh))

    # Thêm chuỗi rỗng vào các mảng có độ dài ngắn hơn
    while len(soNha) < max_length:
        soNha.append(None)
    while len(quanHuyen) < max_length:
        quanHuyen.append(None)
    while len(tinhThanh) < max_length:
        tinhThanh.append(None)
    # Tạo danh sách các từ điển chứa thông tin địa chỉ
    diaDiemLamViec = []
    for a, b, c in zip(soNha, quanHuyen, tinhThanh):
        IdTinhThanh = find_id_tinhThanh(c, df)
        IdQuanHuyen = find_id_tinhQuanHuyen(b, IdTinhThanh[0], df)
        if a == "":
            a = None
    
        diaChiDict = {
            'IdTinhThanh': IdTinhThanh[0],
            'TinhThanh': IdTinhThanh[-1],
            'IdQuanHuyen': IdQuanHuyen[0],
            'TinhQuanHuyen':IdQuanHuyen[-1],
            'DiaChi': a
        }
        diaDiemLamViec.append(diaChiDict)

    return diaDiemLamViec


In [28]:
def extract_number(text):
    # Tìm tất cả các số trong chuỗi
    numbers = re.findall(r'\d+', text)
    # Nếu có số, trả về số đầu tiên (hoặc có thể thay đổi để trả về tất cả nếu cần)
    return int(numbers[0]) if numbers else None

In [29]:
import numpy as np
# Hàm để xử lý kiểu dữ liệu không serializable
def default_converter(o):
    if isinstance(o, (np.integer, np.floating)):  # Xử lý kiểu số NumPy
        return int(o)
    elif isinstance(o, (np.ndarray,)):           # Xử lý mảng NumPy (nếu có)
        return o.tolist()
    return str(o)  # Xử lý các kiểu dữ liệu khác

In [30]:
import re

def extract_schedule(schedule_text):
    # Thay "Chủ nhật" thành "Thứ 8" trong chuỗi đầu vào
    schedule_text = schedule_text.replace("Chủ nhật", "Thứ 8")
    
    # Regex để bắt thông tin
    pattern = r"(?P<dayfrom>Thứ (\d+))\s*-\s*(?P<dayto>Thứ (\d+))\s*\(từ\s*(?P<hourfrom>\d{2}):(?P<minutesfrom>\d{2})\s*đến\s*(?P<hourto>\d{2}):(?P<minutesto>\d{2})\)"
    
    # Thêm một pattern mới để xử lý "Thứ 7" mà không có "- Thứ X"
    pattern_single_day = r"(?P<dayfrom>Thứ (\d+))\s*\(từ\s*(?P<hourfrom>\d{2}):(?P<minutesfrom>\d{2})\s*đến\s*(?P<hourto>\d{2}):(?P<minutesto>\d{2})\)"

    # Kiểm tra trường hợp có "Thứ X - Thứ Y" trước
    match = re.search(pattern, schedule_text)
    
    if not match:  # Nếu không tìm thấy, kiểm tra trường hợp "Thứ X"
        match = re.search(pattern_single_day, schedule_text)
        if match:
            return {
                "dayfrom": int(match.group(2)),  # Chỉ lấy số trong "Thứ X"
                "hourfrom": int(match.group("hourfrom")),
                "minutesfrom": int(match.group("minutesfrom")),
                "dayto": int(match.group(2)),  # dayto và dayfrom giống nhau
                "hourto": int(match.group("hourto")),
                "minutesto": int(match.group("minutesto"))
            }
    
    if match:
        return {
            "dayfrom": int(match.group(2)),  # Chỉ lấy số trong "Thứ X"
            "hourfrom": int(match.group("hourfrom")),
            "minutesfrom": int(match.group("minutesfrom")),
            "dayto": int(match.group(4)),  # Chỉ lấy số trong "Thứ X"
            "hourto": int(match.group("hourto")),
            "minutesto": int(match.group("minutesto"))
        }
    return None


In [31]:
def process_times_data(ThoiGianLamViec):
    ThoiGianCuThe = []
    ThoiGianTuNhap = ""
    data = ThoiGianLamViec.split(';')
    for temp in data:
        if extract_schedule(temp):
            ThoiGianCuThe.append(extract_schedule(temp))
        else:
            if ThoiGianTuNhap:
                ThoiGianTuNhap += "; " + temp
            else:
                ThoiGianTuNhap = temp
    if ThoiGianTuNhap == "":
        ThoiGianTuNhap = None
    dict = {
        'ThoiGianCuThe': ThoiGianCuThe,
        'ThoiGianTuNhap': ThoiGianTuNhap,
    }
    return dict

In [32]:
def get_jobid(soup):
    temp = get_url(soup)
    if temp:
        # Regex
        pattern = r"\/(\d+)(?=\.html)"
        match = re.search(pattern, temp)
        if match:
            return match.group(1)
    return None

In [33]:
def get_meta_description(soup):
    data = soup.find('meta', attrs={"name": "description"})
    if data:
        return str(data.get('content').strip())
    return None

In [34]:
def get_meta_keywords(soup):
    data = soup.find('meta', attrs={"name": "keywords"})
    if data:
        return str(data.get('content').strip())
    return None

In [35]:
def KeywordNganhNghe(soup):
    data = soup.find_all('div', class_="job-tags")
    list = []
    if data:
        for temp in data:
            output = temp.get_text(separator=', ', strip=True).split(',')
            list = list + output
    if list:
        list[0] = process_ChuyenMon(list[0])
        list[:] = filter(None, list)

                
    return list

In [36]:
def thongTinLienQuan(soup):
    titles_to_find = ["Danh mục Nghề liên quan", "Khu vực", 'Kỹ năng cần có', 'Kỹ năng nên có']
    dict = {}
    for title in titles_to_find:
        array = []
        jobs = soup.find('div', class_="box-title", string=title)
        if jobs:
            jobs_div = jobs.find_next('div', class_='box-category-tags')
            if jobs_div:
                # Lấy nội dung của các nghề và loại bỏ khoảng trắng thừa
                jobs = jobs_div.get_text(separator='; ', strip=True).split(';')
                array = array + jobs 
        dict[convert_to_camel_case(title)] = array
    if not dict:
        return None
    return dict

In [ ]:
import random
import requests
# 1369794, step 2 1371126
# 1369795 , step 2, 1371097
df = load_df()
output = []
link_cong_ty_save = []
for temp in range(0,100000,2):
    while True:
        try:
            ids = 1369795 + temp
            scraper = cloudscraper.create_scraper(
                browser={
                    "browser": "chrome",
                    "platform": "windows",
                },
            )

            # specify the target URL

            url = f"https://www.topcv.vn/viec-lam/pho-bo-phan-thiet-ke-kien-truc-noi-that/{ids}.html?ta_source=BoxFeatureJob_LinkDetail"
            # url = 'https://www.topcv.vn/viec-lam/ky-su-kinh-doanh-thiet-bi-moi-truong-tp-hcm-luong-cung-15-000-000-vnd/1367302.html'
            # time.sleep(2)
            response = scraper.get(url)
            print(response.status_code)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                thong_tin_top = thongTinTop(soup)
                # print(thong_tin_top)
                if thong_tin_top:
                    han_nop = hanNopHoSo(soup)
                    linh_vuc_cty = linhVucCongTy(soup)
                    thong_tin_chung = thongTinChung(soup)
                    thong_tin_lien_quan = thongTinLienQuan(soup)
                    linh_vuc = linhVuc(soup)
                    thoi_gian_lam_viec = thoiGianLamViec(soup)
                    ung_tuyen = cachThucUngTuyen(soup)
                    dia_chi = process_address_data(diaChi(soup), df)
                    thong_tin_cong_viec = thongTinCongViec(soup)
                    ten_cong_ty = get_name_company(soup)
                    url_cong_ty = get_link_company(soup)
                    id_cong_ty = get_id_company(soup)
                    
                    muc_luong = thong_tin_top[0].split('-')
                    if len(muc_luong) > 1:
                        muc_luong_den = muc_luong[1].strip()
                        temp_salary = muc_luong_den.split(' ')
                        muc_luong_den = temp_salary[0]
                        muc_luong_tu = muc_luong[0]
                        don_vi_tien = temp_salary[-1]
                        if don_vi_tien.strip().lower() == 'triệu':
                            don_vi_tien = 'VND'
                    else:
                        if muc_luong[0].strip().lower().startswith('trên'):
                            muc_luong_den = 0
                            pattern = r"\d+(\.\d+)?(?=\s*(triệu|usd))"
                            match = re.search(pattern, muc_luong[0].strip().lower())
                            muc_luong_tu = match.group()
                            temp_salary = muc_luong[0].strip().split(' ')
                            don_vi_tien = temp_salary[-1]
                            if don_vi_tien.strip().lower() == 'triệu':
                                don_vi_tien = 'VND'
                        elif muc_luong[0].strip().lower().startswith('dưới'):
                            muc_luong_tu = 0
                            pattern = r"\d+(\.\d+)?(?=\s*(triệu|usd))"
                            match = re.search(pattern, muc_luong[0].strip().lower())
                            muc_luong_den = match.group()
                            temp_salary = muc_luong[0].strip().split(' ')
                            don_vi_tien = temp_salary[-1]
                            if don_vi_tien.strip().lower() == 'triệu':
                                don_vi_tien = 'VND'
                        else:
                            muc_luong_tu = 0
                            muc_luong_den = 0
                            don_vi_tien = None
                    
                    data = {
                        'jobid': int(ids),
                        'jobtitle': get_heading(soup),
                        'urljob': get_url(soup),
                        'mucluongfrom': convert_number(muc_luong_tu),
                        'mucluongto': convert_number(muc_luong_den),
                        'currency': don_vi_tien,
                        'DiaDiem': str(thong_tin_top[1]),
                        'KinhNghiem': str(thong_tin_top[2]),
                        'HanNopHoSo': han_nop,
                        'ChuyenMon': process_ChuyenMon(linh_vuc),
                        'MoTaCongViec': str(thong_tin_cong_viec[0]),
                        'YeuCauUngVien': str(thong_tin_cong_viec[1]),
                        'QuyenLoi': str(thong_tin_cong_viec[2]),
                        'DiaDiemLamViec': dia_chi,
                        'ThoiGianLamViec': process_times_data(str(thoi_gian_lam_viec)),
                        'CapBac': str(thong_tin_chung[0]),
                        'SoLuongTuyen': extract_number(str(thong_tin_chung[1])),
                        'HinhThucLamViec': clean_lines(str(thong_tin_chung[2])),
                        'GioiTinh': str(thong_tin_chung[3]),
                        'IdCongTy': int(id_cong_ty), 
                        'UrlCongTy': str(url_cong_ty),
                        'MetaKeywords': get_meta_keywords(soup),
                        'MetaDescription': get_meta_description(soup),
                        'KeywordNganhNghe': KeywordNganhNghe(soup),
                        'KeywordNganhNgheLienQuan': thong_tin_lien_quan.get('DanhMucNgheLienQuan'),
                        'KeywordKhuVuc': thong_tin_lien_quan.get('KhuVuc'),
                        'KyNangCanCo': thong_tin_lien_quan.get('KyNangCanCo'),
                        'KyNangNenCo': thong_tin_lien_quan.get('KyNangNenCo'),
                    }
                    output.append(data)
                    link_cong_ty_save.append(url_cong_ty)
                    print(json.dumps(data, ensure_ascii=False, indent=5, default=default_converter))  
                    print(response.status_code, "Cập nhật thông tin thành công.", ids, len(output))
            if response.status_code == 429:
                # print(f"ID {ids}: Too many requests, retrying...")
                time.sleep(random.randint(1, 5))  # Chờ một khoảng thời gian trước khi thử lại
            else:
                break  # Thoát khỏi vòng lặp nếu không phải 429
        except (requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
            print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
            time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại
        except (requests.exceptions.ChunkedEncodingError, requests.exceptions.Timeout) as e:
            print(f"ID {ids}: Lỗi kết nối {e}. Đang thử lại...")
            time.sleep(random.randint(1, 5))  # Chờ thêm trước khi thử lại
            


200


NameError: name 'thongTinTop' is not defined

In [ ]:
# Chuyển mảng thành DataFrame
df = pd.DataFrame({'URL Công Ty': link_cong_ty_save})

# Lưu DataFrame vào file Excel
file_path = 'url_cong_ty_save_new3.xlsx'
df.to_excel(file_path, index=False, engine='openpyxl')

print(f"Dữ liệu đã được lưu vào file {file_path}")

Dữ liệu đã được lưu vào file url_cong_ty_save_new3.xlsx


In [ ]:
# Lưu vào tệp JSON
with open("data_2_12_sample_baiViet_topCV_new3.json", "w", encoding="utf-8") as json_file:
    json.dump(output, json_file, ensure_ascii=False, indent=5, default=default_converter)

In [ ]:
len(output)

352

In [ ]:
for temp in output:
    if temp.get('KeywordNganhNghe'):
        print(temp.get('KeywordNganhNghe'))

['Tuyển dụng']
['Chuyên viên thuế']
['Thiết kế khác']
['Sales bất động sản/Môi giới bất động sản']
['Kinh doanh sàn thương mại điện tử']
['Content Marketing']
['Kiểm soát chất lượng (QC)']
['Kinh doanh sàn thương mại điện tử']
['Bán hàng mỹ phẩm']
['Kinh doanh sàn thương mại điện tử']
['Chăm sóc khách hàng']
['Thiết kế khác']
['Content Marketing']
['Head of Marketing/Chief Marketing Officer (CMO)', ' Marketing / Quảng cáo', ' Báo chí']
['Tư vấn du học']
['Quản lý giảng dạy']
['Digital Marketing']
['Digital Marketing']
['Kỹ sư tự động hoá', ' Điện / Điện tử / Điện lạnh', ' Cơ khí / Tự động hóa', ' Sản xuất']
['Digital Marketing']
['Bán hàng siêu thị/cửa hàng tiện ích/tiện lợi']
['Kỹ sư hiện trường']
['Trưởng kho/Giám sát kho']
['Bất động sản/Xây dựng khác']
['Pha chế']
['Kế toán thanh toán', ' Kế toán', ' Bán lẻ - Hàng tiêu dùng - FMCG']
['Kế toán tài sản cố định', ' Kế toán']
['Game Design']
['UI/UX Design']
['Đào tạo']
['Sales Bán lẻ/Dịch vụ tiêu dùng khác']
['Digital Marketing']
['Bá

In [ ]:
for temp in output:
    if temp.get('KeywordNganhNgheLienQuan'):
        for temp1 in temp.get('KeywordNganhNgheLienQuan'):
            if 'Kinh doanh'.lower() in temp1.lower():
                print(temp.get('KeywordNganhNgheLienQuan'))
                break

['Kinh doanh / Bán hàng', ' Marketing / Truyền thông / Quảng cáo', ' Tổ chức sự kiện / Quà tặng']
['Kinh doanh / Bán hàng', ' Dịch vụ khách hàng', ' Quản lý điều hành']
['Kinh doanh / Bán hàng', ' Logistics', ' Xuất nhập khẩu']
['Kinh doanh / Bán hàng', ' Logistics', ' Xuất nhập khẩu']
['Kinh doanh / Bán hàng', ' IT Phần cứng / Mạng', ' IT phần mềm', ' Việc làm IT']
['Kinh doanh / Bán hàng', ' Mỹ phẩm / Trang sức', ' Y tế / Dược']
['Kinh doanh / Bán hàng', ' Kế toán / Kiểm toán']
['Kinh doanh / Bán hàng', ' Quản lý điều hành', ' Dược phẩm / Công nghệ sinh học']
['Kinh doanh / Bán hàng']
['Kinh doanh / Bán hàng', ' Y tế / Dược', ' Bán hàng kỹ thuật']
['Kinh doanh / Bán hàng', ' Tổ chức sự kiện / Quà tặng', ' Bán lẻ / bán sỉ']
['Kinh doanh/Bán hàng', ' Bất động sản/Xây dựng', ' Sales Bất động sản/Xây dựng', ' Sales bất động sản/Môi giới bất động sản']
['Kinh doanh / Bán hàng']
['Kinh doanh / Bán hàng', ' Quản lý chất lượng (QA/QC)', ' Y tế / Dược']
['Kinh doanh / Bán hàng', ' Bảo hiểm', 

In [ ]:
for temp in output:
    if temp.get('KeywordNganhNgheLienQuan'):
        for temp1 in temp.get('KeywordNganhNgheLienQuan'):
            if 'Công nghệ thông tin'.lower() in temp1.lower() or 'cntt'.lower() in temp1.lower() or 'it'.lower() in temp1.lower():
                print(temp.get('KeywordNganhNgheLienQuan'))
                break

['IT Phần cứng / Mạng', ' IT phần mềm', ' Công nghệ thông tin', ' Việc làm IT']
['Công nghệ Thông tin', ' Thiết kế', ' Game Development', ' Nghệ thuật/Hoạt hình/Game', ' Game Design', ' Việc làm IT']
['Công nghệ Thông tin', ' Game Development', ' Game Developer', ' Việc làm IT']
['IT phần mềm', ' Việc làm IT']
['IT phần mềm', ' Việc làm IT']
['IT phần mềm', ' Marketing / Truyền thông / Quảng cáo', ' Việc làm IT']
['Công nghệ thông tin']
['Bưu chính - Viễn thông', ' IT phần mềm', ' Công nghệ thông tin', ' Việc làm IT']
['Báo chí / Truyền hình', ' IT Phần cứng / Mạng', ' Marketing / Truyền thông / Quảng cáo', ' Việc làm IT']
['Công nghệ thông tin']
['Kinh doanh / Bán hàng', ' Giáo dục / Đào tạo', ' IT phần mềm', ' Việc làm IT']
['Marketing/PR/Quảng cáo', ' Digital Marketing', ' Chuyên môn Digital Marketing']
['Marketing/PR/Quảng cáo', ' Digital Marketing', ' Chuyên môn Digital Marketing']
['Công nghệ Thông tin', ' Thiết kế', ' Software Design', ' Thiết kế Đồ hoạ/Giao diện/Trải nghiệm', '

In [ ]:
for temp in output:
    if temp.get('KeywordNganhNgheLienQuan'):
        for temp1 in temp.get('KeywordNganhNgheLienQuan'):
            if 'Y tế'.lower() in temp1.lower() or 'dược'.lower() in temp1.lower() or 'bác sĩ'.lower() in temp1.lower():
                print(temp.get('KeywordNganhNgheLienQuan'))
                break

['Y tế / Dược', ' Spa / Làm đẹp']
['Kinh doanh / Bán hàng', ' Y tế / Dược', ' Spa / Làm đẹp']
['Kinh doanh / Bán hàng', ' Y tế / Dược', ' Dược phẩm / Công nghệ sinh học']
['Kinh doanh/Bán hàng', ' Dược/Y tế/Sức khoẻ/Công nghệ sinh học', ' Bán lẻ/Dịch vụ đời sống', ' Sales Dược/Y tế/Sức khoẻ/Làm đẹp', ' Dịch vụ làm đẹp', ' Tư vấn thẩm mỹ/Spa']
['Kế toán / Kiểm toán', ' Y tế / Dược', ' Spa / Làm đẹp']
['Giáo dục / Đào tạo', ' Tư vấn', ' Y tế / Dược']
['Kinh doanh / Bán hàng', ' Y tế / Dược']
['Y tế / Dược', ' Spa / Làm đẹp']
['Hoá học / Sinh học', ' Nông / Lâm / Ngư nghiệp', ' Dược phẩm / Công nghệ sinh học']
['Y tế / Dược', ' Dược phẩm / Công nghệ sinh học']
['Y tế / Dược']
['Y tế / Dược']
['Y tế / Dược', ' Dược phẩm / Công nghệ sinh học', ' Spa / Làm đẹp']
['Dịch vụ khách hàng', ' Y tế / Dược', ' Spa / Làm đẹp']
['Y tế / Dược']
['Kinh doanh / Bán hàng', ' Tư vấn', ' Dược phẩm / Công nghệ sinh học']
['Y tế / Dược']
['Kinh doanh / Bán hàng', ' Y tế / Dược', ' Dược phẩm / Công nghệ sinh h

In [ ]:
for temp in output:
    if temp.get('KeywordNganhNgheLienQuan'):
        for temp1 in temp.get('KeywordNganhNgheLienQuan'):
            if 'Dịch vụ'.lower() in temp1.lower() or 'Tiêu dùng'.lower() in temp1.lower():
                print(temp.get('KeywordNganhNgheLienQuan'))
                break

['Báo chí / Truyền hình', ' Dịch vụ khách hàng', ' Marketing / Truyền thông / Quảng cáo']
['Biên / Phiên dịch', ' Dịch vụ khách hàng', ' Xuất nhập khẩu']
['Kinh doanh / Bán hàng', ' Dịch vụ khách hàng', ' Hành chính / Văn phòng']
['Dịch vụ khách hàng', ' Hành chính / Văn phòng', ' Vận tải / Kho vận']
['Kinh doanh / Bán hàng', ' Dịch vụ khách hàng', ' Tư vấn']
['Kinh doanh / Bán hàng', ' Dịch vụ khách hàng']
['Kinh doanh/Bán hàng', ' Dược/Y tế/Sức khoẻ/Công nghệ sinh học', ' Bán lẻ/Dịch vụ đời sống', ' Sales Dược/Y tế/Sức khoẻ/Làm đẹp', ' Dịch vụ làm đẹp', ' Tư vấn thẩm mỹ/Spa']
['Dịch vụ khách hàng', ' Logistics', ' Xuất nhập khẩu']
['Kinh doanh/Bán hàng', ' Bán lẻ/Dịch vụ đời sống', ' Sales Bán lẻ/Dịch vụ tiêu dùng', ' Bán lẻ', ' Bán hàng mỹ phẩm']
['Kinh doanh / Bán hàng', ' Dịch vụ khách hàng', ' Spa / Làm đẹp']
['Kinh doanh / Bán hàng', ' Hàng tiêu dùng']
['Dịch vụ khách hàng', ' Khách sạn / Nhà hàng', ' Marketing / Truyền thông / Quảng cáo']
['Biên / Phiên dịch', ' Dịch vụ khách h